In [1]:
import os

import numpy as np

import matplotlib
matplotlib.use('Agg')           # noqa: E402
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import tensorflow.compat.v1 as tf

from attacks import jsma

from PIL import Image
import time

In [2]:
img_size = 32
img_chan = 3
n_classes = 10

In [3]:
print('\nLoading CIFAR10')

cifar = tf.keras.datasets.cifar10
(X_train, y_train), (X_test, y_test) = cifar.load_data()
print(np.shape(X_train))
X_train = np.reshape(X_train, [-1, img_size, img_size, img_chan])
X_train = X_train.astype(np.float32) / 255
X_test = np.reshape(X_test, [-1, img_size, img_size, img_chan])
X_test = X_test.astype(np.float32) / 255

to_categorical = tf.keras.utils.to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(X_test.shape)
print(y_test.shape)


Loading CIFAR10
(50000, 32, 32, 3)
(10000, 32, 32, 3)
(10000, 10)


In [5]:
print('\nSpliting data')

ind = np.random.permutation(X_train.shape[0])
X_train, y_train = X_train[ind], y_train[ind]

VALIDATION_SPLIT = 0.1
n = int(X_train.shape[0] * (1-VALIDATION_SPLIT))
X_valid = X_train[n:]
X_train = X_train[:n]
y_valid = y_train[n:]
y_train = y_train[:n]


Spliting data


In [6]:
print('\nConstruction graph')


def model(x, logits=False, training=False):

        
    with tf.variable_scope('conv0'):
        z = tf.layers.conv2d(x, filters=32, kernel_size=[3, 3],
                             padding='same', activation=tf.nn.relu)
        z = tf.layers.max_pooling2d(z, pool_size=[2, 2], strides=2)

    with tf.variable_scope('conv1'):
        z = tf.layers.conv2d(z, filters=64, kernel_size=[3, 3],
                             padding='same', activation=tf.nn.relu)
        z = tf.layers.max_pooling2d(z, pool_size=[2, 2], strides=2)

    with tf.variable_scope('flatten'):
        shape = z.get_shape().as_list()
        z = tf.reshape(z, [-1, np.prod(shape[1:])])

    with tf.variable_scope('mlp'):
        z = tf.layers.dense(z, units=128, activation=tf.nn.relu)
        z = tf.layers.dropout(z, rate=0.25, training=training)

    logits_ = tf.layers.dense(z, units=10, name='logits')
    y = tf.nn.softmax(logits_, name='ybar')

    if logits:
        return y, logits_
    return y


class Dummy:
    pass

import tensorflow.compat.v1 as tf
env = Dummy()

tf.disable_eager_execution()
with tf.variable_scope('model', reuse=tf.AUTO_REUSE):
    env.x = tf.placeholder(tf.float32, (None, img_size, img_size, img_chan),
                           name='x')
    env.y = tf.placeholder(tf.float32, (None, n_classes), name='y')
    env.training = tf.placeholder_with_default(False, (), name='mode')
    
    env.ybar, logits = model(env.x, logits=True, training=env.training)

    with tf.variable_scope('acc'):
        count = tf.equal(tf.argmax(env.y, axis=1), tf.argmax(env.ybar, axis=1))
        env.acc = tf.reduce_mean(tf.cast(count, tf.float32), name='acc')

    with tf.variable_scope('loss'):
        xent = tf.nn.softmax_cross_entropy_with_logits(labels=env.y,
                                                       logits=logits)
        env.loss = tf.reduce_mean(xent, name='loss')

    with tf.variable_scope('train_op'):
        optimizer = tf.train.AdamOptimizer()
        env.train_op = optimizer.minimize(env.loss)

    env.saver = tf.train.Saver()

with tf.variable_scope('model', reuse=True):
    env.target = tf.placeholder(tf.int32, (), name='target')
    env.adv_epochs = tf.placeholder_with_default(20, shape=(), name='epochs')
    env.adv_eps = tf.placeholder_with_default(0.2, shape=(), name='eps')
    env.x_jsma = jsma(model, env.x, env.target, eps=env.adv_eps, k=1,
                      epochs=env.adv_epochs)


Construction graph
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Use `tf.cast` instead.


In [7]:
print('\nInitializing graph')

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())


Initializing graph


In [8]:
def evaluate(sess, env, X_data, y_data, batch_size=128):
    """
    Evaluate TF model by running env.loss and env.acc.
    """
    print('\nEvaluating')
    startTime = time.time()

    n_sample = X_data.shape[0]
    n_batch = int((n_sample+batch_size-1) / batch_size)
    loss, acc = 0, 0

    for batch in range(n_batch):
        print(' batch {0}/{1}'.format(batch + 1, n_batch), end='\r')
        start = batch * batch_size
        end = min(n_sample, start + batch_size)
        cnt = end - start
        batch_loss, batch_acc = sess.run(
            [env.loss, env.acc],
            feed_dict={env.x: X_data[start:end],
                       env.y: y_data[start:end]})
        loss += batch_loss * cnt
        acc += batch_acc * cnt
    loss /= n_sample
    acc /= n_sample
    endTime = time.time()

    print(' loss: {0:.4f} acc: {1:.4f}'.format(loss, acc))
    print(endTime - startTime)
    return loss, acc


def train(sess, env, X_data, y_data, X_valid=None, y_valid=None, epochs=1,
          load=False, shuffle=True, batch_size=128, name='model'):
    """
    Train a TF model by running env.train_op.
    """
    if load:
        if not hasattr(env, 'saver'):
            return print('\nError: cannot find saver op')
        print('\nLoading saved model')
        return env.saver.restore(sess, 'model/{}'.format(name))

    print('\nTrain model')
    n_sample = X_data.shape[0]
    n_batch = int((n_sample+batch_size-1) / batch_size)
    for epoch in range(epochs):
        print('\nEpoch {0}/{1}'.format(epoch + 1, epochs))

        if shuffle:
            print('\nShuffling data')
            ind = np.arange(n_sample)
            np.random.shuffle(ind)
            X_data = X_data[ind]
            y_data = y_data[ind]

        for batch in range(n_batch):
            print(' batch {0}/{1}'.format(batch + 1, n_batch), end='\r')
            start = batch * batch_size
            end = min(n_sample, start + batch_size)
            sess.run(env.train_op, feed_dict={env.x: X_data[start:end],
                                              env.y: y_data[start:end],
                                              env.training: True})
        if X_valid is not None:
            evaluate(sess, env, X_valid, y_valid)

    if hasattr(env, 'saver'):
        print('\n Saving model')
        os.makedirs('model', exist_ok=True)
        env.saver.save(sess, 'model/{}'.format(name))


def predict(sess, env, X_data, batch_size=128):
    """
    Do inference by running env.ybar.
    """
    print('\nPredicting')
    n_classes = env.ybar.get_shape().as_list()[1]

    n_sample = X_data.shape[0]
    n_batch = int((n_sample+batch_size-1) / batch_size)
    yval = np.empty((n_sample, n_classes))

    for batch in range(n_batch):
        print(' batch {0}/{1}'.format(batch + 1, n_batch), end='\r')
        start = batch * batch_size
        end = min(n_sample, start + batch_size)
        y_batch = sess.run(env.ybar, feed_dict={
            env.x: X_data[start:end],
            env.target: np.random.choice(n_classes)})
        print(y_batch)
        yval[start:end] = y_batch
    print()
    return yval


# To do the boundary analysis for adversarial samples
def boundary_analysis(sess, env, X_data, y_data, batch_size=128):
    """
    Evaluate TF model by running env.loss and env.acc.
    """
    print('\nPredicting')
    n_classes = env.ybar.get_shape().as_list()[1]

    n_sample = X_data.shape[0]
    n_batch = int((n_sample+batch_size-1) / batch_size)
    yval = np.empty((n_sample, n_classes))

    img_count = 0
    diff_confidence_score_total = 0.0
    for batch in range(n_batch):
        #print(' batch {0}/{1}'.format(batch + 1, n_batch), end='\r')
        start = batch * batch_size
        end = min(n_sample, start + batch_size)
        y_batch = sess.run(env.ybar, feed_dict={
            env.x: X_data[start:end],
            env.target: np.random.choice(n_classes)})
        for yy in y_batch:
            yy2 = yy.tolist()
            yy2.sort(reverse=True)
            first_confidence_score = yy2[0]
            second_confidence_score = yy2[1]
            diff_confidence_score = first_confidence_score - second_confidence_score
            diff_confidence_score_total += diff_confidence_score
            img_count += 1
        yval[start:end] = y_batch
    print(diff_confidence_score_total / img_count)
    print()
    return yval


def make_jsma(sess, env, X_data, epochs=0.2, eps=1.0, batch_size=128):
    """
    Generate JSMA by running env.x_jsma.
    """
    print('\nMaking adversarials via JSMA')

    startTime = time.time()
    n_sample = X_data.shape[0]
    n_batch = int((n_sample + batch_size - 1) / batch_size)
    X_adv = np.empty_like(X_data)

    for batch in range(n_batch):
        print(' batch {0}/{1}'.format(batch + 1, n_batch), end='\r')
        start = batch * batch_size
        end = min(n_sample, start + batch_size)
        feed_dict = {
            env.x: X_data[start:end],
            env.target: np.random.choice(n_classes),
            env.adv_epochs: epochs,
            env.adv_eps: eps}
        adv = sess.run(env.x_jsma, feed_dict=feed_dict)
        X_adv[start:end] = adv
    endTime = time.time()
    print()
    print(endTime - startTime)

    return X_adv

In [9]:
print('\nTraining')

train(sess, env, X_train, y_train, X_valid, y_valid, load=True, epochs=5,
      name='cifar10')


Training

Loading saved model
INFO:tensorflow:Restoring parameters from model/cifar10


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key model/train_op/model/conv0/conv2d/bias/Adam not found in checkpoint
	 [[node model/save/RestoreV2 (defined at <ipython-input-6-5a9d998410f7>:61) ]]

Original stack trace for 'model/save/RestoreV2':
  File "/Users/i537199/opt/anaconda3/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/i537199/opt/anaconda3/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/i537199/opt/anaconda3/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/i537199/opt/anaconda3/lib/python3.8/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/Us

In [8]:
print('\nEvaluating on clean data')

evaluate(sess, env, X_test, y_test)


Evaluating on clean data

Evaluating
 loss: 2.3052 acc: 0.0997
5.4840922355651855


(2.3051980388641358, 0.0997)

In [10]:
print('\nGenerating adversarial data')

X_adv = make_jsma(sess, env, X_train, epochs=30, eps=0)
X_adv1 = make_jsma(sess, env, X_train, epochs=30, eps=0.1)
#X_adv2 = make_jsma(sess, env, X_test, epochs=30, eps=0.2)
#X_adv3 = make_jsma(sess, env, X_test, epochs=30, eps=0.3)
#X_adv4 = make_jsma(sess, env, X_test, epochs=30, eps=0.4)
#X_adv5 = make_jsma(sess, env, X_test, epochs=30, eps=0.5)


Generating adversarial data

Making adversarials via JSMA
 batch 352/352
924.8766701221466

Making adversarials via JSMA
 batch 352/352
989.5228879451752


In [12]:
i = 0
for imageArr in X_adv1:
    im = Image.fromarray((imageArr * 255).astype(np.uint8))
    im.save("jsma_perturbed_images/0.1/" + str(i) + ".png")
    i += 1

In [17]:
print('\nEvaluating on adversarial data')

evaluate(sess, env, X_adv, y_test)
evaluate(sess, env, X_adv1, y_test)
evaluate(sess, env, X_adv2, y_test)
evaluate(sess, env, X_adv3, y_test)
evaluate(sess, env, X_adv4, y_test)
evaluate(sess, env, X_adv5, y_test)


Evaluating on adversarial data

Evaluating
 loss: 2.3188 acc: 0.1062
2.4008052349090576

Evaluating
 loss: 2.3190 acc: 0.1044
2.2438721656799316

Evaluating
 loss: 2.3192 acc: 0.1055
2.197758913040161

Evaluating
 loss: 2.3193 acc: 0.1051
2.3111629486083984

Evaluating
 loss: 2.3195 acc: 0.1035
2.2774298191070557

Evaluating
 loss: 2.3193 acc: 0.1048
2.5419719219207764


(2.319290768814087, 0.1048)

In [ ]:
print('\nDoing boundary analysis on adversarial data')

print('\nDifference in confidence score when eps = 0.0')
boundary_analysis(sess, env, X_adv, y_test)
print('\nDifference in confidence score when eps = 0.1')
boundary_analysis(sess, env, X_adv1, y_test)
print('\nDifference in confidence score when eps = 0.2')
boundary_analysis(sess, env, X_adv2, y_test)
print('\nDifference in confidence score when eps = 0.3')
boundary_analysis(sess, env, X_adv3, y_test)
print('\nDifference in confidence score when eps = 0.4')
boundary_analysis(sess, env, X_adv4, y_test)
print('\nDifference in confidence score when eps = 0.5')
boundary_analysis(sess, env, X_adv5, y_test)

In [ ]:
print('\nRandomly sample adversarial data from each category')

z0 = np.argmax(y_test, axis=1)
z1 = np.argmax(predict(sess, env, X_test), axis=1)
ind = z0 == z1

X_data = X_test[ind]
labels = z0[ind]

X_adv = np.empty((10, 10, 32, 32, 3))

for source in np.arange(10):
    print('Source label {0}'.format(source))

    X_i = X_data[labels == source]

    for i, xi in enumerate(X_i):
        found = True
        xi = xi[np.newaxis, :]

        for target in np.arange(10):
            print(' [{0}/{1}] {2} -> {3}'
                  .format(i+1, X_i.shape[0], source, target), end='')

            if source == target:
                xadv = xi.copy()
            else:
                feed_dict = {env.x: xi, env.target: target, env.adv_epochs: 30,
                             env.adv_eps: 0.12}
                xadv = sess.run(env.x_jsma, feed_dict=feed_dict)

            yadv = predict(sess, env, xadv)
            label = np.argmax(yadv.flatten())
            found = target == label

            if not found:
                print(' Fail')
                break

            X_adv[source, target] = np.squeeze(xadv)
            print(' res: {0} {1:.2f}'.format(label, np.max(yadv)))

        if found:
            break

In [ ]:
print('\nGenerating figure')

fig = plt.figure(figsize=(10, 10))
gs = gridspec.GridSpec(10, 10, wspace=0.1, hspace=0.1)

for i in range(10):
    for j in range(10):
        ax = fig.add_subplot(gs[i, j])
        ax.imshow(X_adv[i, j], cmap='gray', interpolation='none')
        ax.set_xticks([])
        ax.set_yticks([])

        if i == j:
            for spine in ax.spines:
                ax.spines[spine].set_color('green')
                ax.spines[spine].set_linewidth(5)

        if ax.is_first_col():
            ax.set_ylabel(i, fontsize=20, rotation='horizontal', ha='right')
        if ax.is_last_row():
            ax.set_xlabel(j, fontsize=20)

gs.tight_layout(fig)
os.makedirs('img', exist_ok=True)
plt.savefig('img/jsma_cifar10_diff.png')